# GlobustVP Demo

This notebook demonstrates how to use **GlobustVP** to estimate vanishing points from a real image.

In [ ]:
# 📦 Setup (only needed if running on Google Colab or outside the repo)
from pathlib import Path

if Path.cwd().name != "GlobustVP":
    !git clone --quiet https://github.com/WU-CVGL/GlobustVP.git
    %cd GlobustVP
    !pip install --quiet --progress-bar off -e .

# 🔁 (Optional) Reload modules automatically if you modify the source code
# %load_ext autoreload
# %autoreload 2

## 🔧 Imports

We import the necessary modules from the GlobustVP package.

In [ ]:
import time
import numpy as np

from globustvp.solver.core import globustvp
from globustvp.utils.io import load_image_and_gray
from globustvp.utils.geometry import (
    normalize_lines,
    compute_backprojection_normals,
    compute_line_uncertainties
)
from globustvp.utils.plot import plot_lines_on_image, visualize_line_vp_associations
from globustvp.utils.line_processing import detect_and_format_lines

## 📷 Step 1: Load input image
Read the target image and convert to grayscale.

In [ ]:
print("📷 Loading image...")
img, gray = load_image_and_gray("assets/indoor.jpg")

## 🧪 Step 2: Detect and filter line segments using LSD
We use LSD detector, filter lines by length, and perform clustering based on dominant orientation.

In [ ]:
print("🧪 Running LSD line detection and filtering...")
lines_2D = detect_and_format_lines(gray_image=gray, min_length=30)
print(f"✅ {lines_2D.shape[1]} lines remain after filtering.")

## 📊 Step 3: Visualize detected line segments

In [ ]:
plot_lines_on_image(image=img, lines=lines_2D)

## 📐 Step 4: Normalize line segments using intrinsics

In [ ]:
print("📐 Normalizing lines using camera intrinsics...")
K = np.array([[674.917975164175, 0, 307.551305282635],
              [0, 674.917975164175, 251.454244960136],
              [0, 0, 1]])

normalized_lines = normalize_lines(K, lines_2D).T
print("✅ Normalization complete.")

## 🧮 Step 5: Convert normalized lines to plane normals

In [ ]:
print("🧮 Converting lines to back-projection normals...")
para_lines = compute_backprojection_normals(normalized_lines)
print("✅ Generated", para_lines.shape[0], "normals.")

## 📏 Step 6: Estimate uncertainty for each line

In [ ]:
print("📏 Assigning uncertainty weights...")
uncertainty = compute_line_uncertainties(normalized_lines, K, use_uncertainty=True)

## 🚀 Step 7: Run GlobustVP solver

In [ ]:
param = {
    "line_num": lines_2D.shape[1],
    "vanishing_point_num": 3,
    "c": 0.03,
    "sample_line_num": 4,
    "is_fast_solver": True,
    "eigen_threshold": 1,
    "solver": "SCS",
    "solver_opts": {"eps_abs": 1e-12, "eps_rel": 1e-12},
    "K": K
}

print("🚀 Running GlobustVP solver...")
t_start = time.time()
status, est_vps, est_corrs = globustvp(normalized_lines, para_lines, uncertainty, param)
t_end = time.time()

if status:
    print("✅ GlobustVP succeeded in {:.4f} seconds".format(t_end - t_start))
    print("🧭 Estimated vanishing points:\n", est_vps)
else:
    print("❌ GlobustVP failed to converge.")

## 🎨 Step 8: Visualize line-VP association results

In [ ]:
print("🎨 Plotting vanishing point estimation result...")
visualize_line_vp_associations(image=img, lines=lines_2D, est_corrs=est_corrs)